In [ ]:
!pip install -q lightning
!pip install -q transformers
!pip install -q torchmetrics
!pip install -q watermark
!pip install -q torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import time
import random
import warnings
import os.path as op
# from google.colab import drive
# drive.mount("/content/drive")

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

import lightning as L
from watermark import watermark
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint,EarlyStopping

import torch
import torchmetrics
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchinfo import summary
from torch.utils.data import WeightedRandomSampler,Dataset, DataLoader,TensorDataset

import transformers
from transformers import (
    AdamW,
    WarmUp,
    get_linear_schedule_with_warmup,
    DistilBertTokenizer,
    DistilBertModel
)

import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')

SEED=42
warnings.filterwarnings(action="ignore",category=UserWarning)
%matplotlib inline

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     d:\Anaconda3\envs\nlpProjects\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     d:\Anaconda3\envs\nlpProjects\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     d:\Anaconda3\envs\nlpProjects\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
def seed_everything(seed):
  random.seed(seed)
  os.environ["PYTHONHASHSEED"]=str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark=True
seed_everything(SEED)

In [4]:
class Config:
  batch_size=32
  num_workers=4
  lr=0.00003
  epochs=200
  load_weights_path="models/"
  save_file_name="model_weights_distilbert_lightning_v1"
  MODEL_NAME="distilbert-base-uncased"
  device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def get_model():
  tokenizer=DistilBertTokenizer.from_pretrained(Config.MODEL_NAME)
  SPECIAL_TOKENS_DICT={
     'sep_token':'[SEP]', 
     'pad_token':'[PAD]',
     'cls_token':'[CLS]' 
  }
  tokenizer.add_special_tokens(SPECIAL_TOKENS_DICT)
  model=DistilBertModel.from_pretrained(Config.MODEL_NAME)
  model.resize_token_embeddings(len(tokenizer))
  return model,tokenizer

## EDA

In [17]:
dataset=pd.read_csv("../data/annotated_data/Train.csv",encoding='UTF-8')
dataset.head()

,Text,Label
0,"Dès les premières pages, on peut constater un ...",None
1,@maxvandewiele penser qu’il y a trop d’immigré...,None
2,@dv.antoine c'est magnifique ce que tu dis par...,Bullying
3,@maxvandewiele en quoi?,None
4,@dv.antoine content que tu pose la question : ...,None


In [18]:
test_df=pd.read_csv("../data/annotated_data/test.csv",encoding='UTF-8')
test_df.head()

,Text
0,"Encore une fois je ne dis pas le contraire, c'..."
1,En même temps la bas c'est un sketch des fois....
2,"Dites, la modération à changé les règles pour ..."
3,Je vais suivre l'élection. Il y a quelques ann...
4,Je ne pense pas qu'il y ait de racisme structu...


In [19]:
def clean_text(data):
    """
    input: data: a dataframe containing texts to be cleaned
    return: the same dataframe with an added column of clean text
    """
    clean_data = data.copy()
    clean_data = clean_data[['Text', 'Label']]
    stop_words = stopwords.words('french')
    clean_text = []
    tokenizer = RegexpTokenizer(r'\w+')
    lemmatizer = WordNetLemmatizer()

    for idx in range(len(data)):
        text = clean_data['Text'][idx]
        label = clean_data['Label'][idx]
        text_lowercase = text.lower()
        text_no_stopwords = " ".join([word for word in text_lowercase.split() if word not in (stop_words)])
        text_tokenized = tokenizer.tokenize(text_no_stopwords)
        text_lemmatized = [lemmatizer.lemmatize(token) for token in text_tokenized]
        clean_text = ' '.join(text_lemmatized)
        clean_data['Text'][idx] = clean_text
    return clean_data

In [20]:
dataset=clean_text(dataset)
dataset["Label"].replace("None","none",inplace=True)
dataset["Label"].replace("none ","none",inplace=True)
dataset["Label"].replace("Sexism","sexism",inplace=True)
dataset["Label"].replace("sexism ","sexism",inplace=True)
dataset["Label"].replace("Sexism ","sexism",inplace=True)
dataset["Label"].replace("Homophobia","homophobia",inplace=True)
dataset["Label"].replace("Hate Speech","Hate_Speech",inplace=True)

In [21]:
dataset['Label'].value_counts()

none           4133
homophobia     1198
Bullying        632
Hate_Speech     484
Racism          256
sexism          214
Name: Label, dtype: int64

In [22]:
labeldict={"none":0,"homophobia":1,"Bullying":2,"Hate_Speech":3,"Racism":4,"sexism":5}
dataset["Label"]=dataset["Label"].map(labeldict)

In [23]:
data=pd.DataFrame()
data["text"]=dataset["Text"]
data["labels"]=dataset.iloc[:,1:].values

In [24]:
data.head()

,text,labels
0,dès premières page peut constater problème mét...,0
1,maxvandewiele penser qu il a trop d immigrés n...,0
2,dv antoine c est magnifique dis parce c est faux,2
3,maxvandewiele quoi,0
4,dv antoine content pose question opinion prône...,0


In [25]:
class LightningSentimentDataset(nn.Module):
    def __init__(self,data,tokenizer,max_len=128):
        self.data=data
        self.tokenizer=tokenizer
        self.max_len=max_len
        self.text=self.data.text
        self.targets=self.data.labels

    def __len__(self):
        return len(self.text)

    def __getitem__(self,index):
        text=str(self.text[index])
        text=" ".join(text.split())

        inputs=self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            pad_to_max_length=True,
            return_token_type_ids=True
        )

        input_ids=inputs["input_ids"]
        attention_mask=inputs["attention_mask"]

        return {
            "input_ids":torch.tensor(input_ids,dtype=torch.long),
            "attention_mask":torch.tensor(attention_mask,dtype=torch.long),
            "targets":torch.tensor(self.targets[index],dtype=torch.long)
        }
    
    def __len__(self):
        return len(self.text)

In [26]:
#@title I'll be back later
class DistilBertClass(nn.Module):
  def __init__(self,num_features,num_classes):
    super(DistilBertClass,self).__init__()
    self.l1=model,_=get_model()
    self.classifier=nn.Sequential(
        nn.Linear(num_features,768),
        nn.Tanh(),
        nn.Dropout(0.8),
        nn.Linear(768,num_classes)
    )

    def forward(self,input_ids,attention_mask):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      hidden_state=output_1[0]
      pooler=hidden_state[:,0]
      logits=self.classifier(pooler)
      return logits 

In [29]:
class LightningModel(L.LightningModule):
    def __init__(self,learning_rate,num_features,num_classes):
      super(LightningModel,self).__init__()
      self.learning_rate=learning_rate
      self.save_hyperparameters(ignore=['model'])
      self.train_acc=torchmetrics.Accuracy(task="multiclass",num_classes=6)
      self.val_acc=torchmetrics.Accuracy(task="multiclass",num_classes=6)
      self.num_features=num_features
      self.num_classes=num_classes
      self.l1,_=get_model()
      
      self.classifier=nn.Sequential(
          nn.Linear(self.num_features,768),
          nn.Tanh(),
          nn.Dropout(0.8),
          nn.Linear(768,self.num_classes)
        )
        
    def forward(self,input_ids,attention_mask):
      output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
      # print(output_1.last_hidden_state.shape)
      hidden_state=output_1[0]
      # assert output_1.last_hidden_state.shape == output_1[0].shape
      pooler=hidden_state[:,0]
      # assert output_1.last_hidden_state.shape == pooler.shape
      logits=self.classifier(pooler)
      return logits 
    
    def training_step(self,batch,true_labels):
      outputs={
          'input_ids':batch["input_ids"],
          'attention_mask':batch["attention_mask"]
      }
      true_labels=batch["targets"]
      logits=self.forward(**outputs)
      # print(logits.shape)
      loss=nn.CrossEntropyLoss()(logits,true_labels)
      self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
      with torch.no_grad():
        predicted_labels=torch.argmax(logits,dim=1)
        self.train_acc(predicted_labels, true_labels)
        self.log("train_acc", self.train_acc, on_epoch=True, on_step=False)
      return loss
    
    def validation_step(self,batch,true_labels):
      outputs={
          'input_ids':batch["input_ids"],
          'attention_mask':batch["attention_mask"]
      }
      true_labels=batch["targets"]
      logits=self.forward(**outputs)
      loss=nn.CrossEntropyLoss()(logits,true_labels)
      self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
      predicted_labels=torch.argmax(logits,dim=1)
      self.val_acc(predicted_labels,true_labels)
      self.log("val_acc",self.val_acc,prog_bar=True)
    
    def configure_optimizers(self):
      optimizer=torch.optim.AdamW(self.parameters(),lr=self.learning_rate)
      return optimizer

In [30]:
lightning_model=LightningModel(learning_rate=Config.lr,num_features=768,num_classes=6)
lightning_model

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LightningModel(
  (train_acc): MulticlassAccuracy()
  (val_acc): MulticlassAccuracy()
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (

In [16]:
if __name__=='__main__':
  print(watermark(packages="torch,lightning,transformers", python=True), flush=True)
  print("Torch CUDA available?", torch.cuda.is_available(), flush=True)

  ###################### Set up DataLoaders####################
  _,tokenizer=get_model()

  train_size=0.8
  train_data=data.sample(frac=train_size,random_state=seed_everything(SEED))
  test_data=data.drop(train_data.index).reset_index(drop=True)
  train_data=train_data.reset_index(drop=True)
  training_set = LightningSentimentDataset(train_data, tokenizer, 100)
  testing_set = LightningSentimentDataset(test_data, tokenizer, 100)

  train_loader=DataLoader(
    training_set,
    batch_size=Config.batch_size,
    num_workers=Config.num_workers)

  val_loader=DataLoader(
      testing_set,
      batch_size=Config.batch_size,
      num_workers=Config.num_workers)
  
  lightning_model=LightningModel(learning_rate=Config.lr,num_features=768,num_classes=6)
  summary(lightning_model)
  
  callbacks = [
        ModelCheckpoint(save_top_k=1, mode="max", monitor="val_acc"),  # save top 1 model
        EarlyStopping(monitor="val_acc", min_delta=0.00, patience=3, verbose=False, mode="max")
    ]
  logger=CSVLogger(save_dir="logs/",name=Config.save_file_name)

  trainer=L.Trainer(
      max_epochs=Config.epochs,
      callbacks=callbacks,
      accelerator="gpu",
      precision="16-mixed",
      logger=logger,
      log_every_n_steps=10,
      )
  
  start = time.time()
  trainer.fit(lightning_model,train_loader,val_loader)

  end=time.time()
  elapsed = end - start
  print(f"Time elapsed {elapsed/60:.2f} min")

  # test_acc = trainer.test(lightning_model, dataloaders=val_loader, ckpt_path="best")
  # print(test_acc)
  val_acc=trainer.validate(lightning_model, dataloaders=val_loader, ckpt_path="best")
  print(val_acc)

  with open(op.join(trainer.logger.log_dir, "outputs.txt"), "w") as f:
    f.write((f"Time elapsed {elapsed/60:.2f} min\n"))
    f.write(f"Test acc: {val_acc}")


Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

torch       : 2.0.0+cu117
lightning   : 2.0.1
transformers: 4.27.4

Torch CUDA available? True


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- T

Sanity Checking: 0it [00:00, ?it/s]

In [ ]:
_,tokenizer=get_model()
def predict(text):
    encoded_text=tokenizer.encode_plus(
        text,
        max_length=100,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids=encoded_text['input_ids']
    attention_mask=encoded_text['attention_mask']
    model=lightning_model.load_from_checkpoint(checkpoint_path="src/SentimentAnalysis/logs/model_weights_distilbert_lightning_v1/version_0/checkpoints/distilBERT_v1.ckpt.ckpt").to('cpu')
    model.eval()
    with torch.no_grad():
      output=model(input_ids,attention_mask).to('cpu')
      probabilities=torch.softmax(output,dim=1)
      confidence, predicted_class=torch.max(probabilities,dim=1)
      key = list(filter(lambda x: labeldict[x] == predicted_class, labeldict))[0]
      print("---------------------------RESULT------------------------------")
      print(f'Predicted class:{predicted_class}, Confidence:{confidence}, label:{key}')

predict("""En même temps la bas c'est un sketch des fois. Après une bagarre parce que des cons de gitans aient embrouillé un 
pote, ils ont suivi d'autres potes qui n'avaient rien a voir la dedans en leur courant après, du coup on commence à leur courrir 
après, une voiture s'arrête à leur hauteur, deux maghrébins sortent, le plus chaud des gitans prends une turbo gifle, son pote s'arrête 
net, le gifleur leur dit d'arrêter les conneries puis s'en va. Après qu'ils soient partis le gitan leur crie 'sale arabe'./r\nSuper fête
 de la musique.""")